In [97]:
import os
import lancedb
import polars as pl

#print(filenames,len(filenames))

In [107]:
#from ipywidgets import interact, interactive, fixed, interact_manual, controls
import ipywidgets
from sentence_transformers import SentenceTransformer, util, InputExample, losses
from langchain_text_splitters import RecursiveCharacterTextSplitter
model = SentenceTransformer("thenlper/gte-base")

config="00"
met="cosine"

filenames = os.listdir("../config_%s"%config)   #Not sure why .DS_Store shows up in some folders, if not then comment out the .remove line
print(filenames)
if '.DS_Store' in filenames:
   filenames.remove('.DS_Store')
print(filenames,len(filenames))



['vectorized_config_00_data_pt7.parquet', '.DS_Store', 'vectorized_config_00_data_pt6.parquet', 'Config_00_lancedb', 'vectorized_config_00_data_pt4.parquet', 'query_results_config_00_metric_L2_full.parquet', 'vectorized_config_00_data_pt5.parquet', 'query_results_config_00_metric_cosine_full.parquet', 'vectorized_config_00_data_pt9.parquet', 'vectorized_config_00_data_pt0.parquet', 'vectorized_config_00_data_pt1.parquet', 'vectorized_config_00_data_pt8.parquet', 'vectorized_config_00_data_pt3.parquet', 'vectorized_config_00_data_pt10.parquet', 'vectorized_config_00_data_pt2.parquet']
['vectorized_config_00_data_pt7.parquet', 'vectorized_config_00_data_pt6.parquet', 'Config_00_lancedb', 'vectorized_config_00_data_pt4.parquet', 'query_results_config_00_metric_L2_full.parquet', 'vectorized_config_00_data_pt5.parquet', 'query_results_config_00_metric_cosine_full.parquet', 'vectorized_config_00_data_pt9.parquet', 'vectorized_config_00_data_pt0.parquet', 'vectorized_config_00_data_pt1.parque

In [108]:
## Initialize a database
db = lancedb.connect("../config_%s/Config_%s_lancedb"%(config,config))

In [109]:
filenames_data = []
for i in range(len(filenames)):
    if filenames[i].startswith('vectorized_config'):
       filenames_data.append(filenames[i]) 
#print(filenames_data)

In [ ]:
%%time

## Load the first data file
data = pl.read_parquet("../config_%s/vectorized_config_%s_data_pt0.parquet"%(config,config))
## Create a table in the database based on the first file, ignore warning if the table already exists. Use mode="overwrite" if you want scorched earth
table = db.create_table("Config_%s_table"%config, data=data,mode="overwrite")
    
## Load the remaining data into the table, one file at a time, there are 11 of them in total

for i in range(1,len(filenames_data)):
    data = pl.read_parquet("../config_%s/vectorized_config_%s_data_pt%s.parquet"%(config,config,str(i)))
    table.add(data)
    print(f"added vectorized_config_{config}_data_pt{i}.parquet to database")
    

added vectorized_config_00_data_pt1.parquet to database
added vectorized_config_00_data_pt2.parquet to database
added vectorized_config_00_data_pt3.parquet to database


### Build ANN Index

In [ ]:
%%time
## Build the ANN Index
## https://lancedb.github.io/lancedb/ann_indexes/#creating-an-ivf_pq-index
table.create_index(met,num_partitions=1024,num_sub_vectors=96,accelerator="mps") ## to use Apple Metal
## It is safe to ignore the UserWarning:...

In [ ]:
import time

def make_query(query_text, num_results=50, probes=20, refine_factor=10):
    
    start_time = time.time()
    ## Embed query
    query = model.encode(query_text)

    ## Perform search with the metric specified by the variable met
    
    result = table.search(query).metric(met).limit(num_results).nprobes(20).refine_factor(10).to_polars()
    end_time = time.time()
    print(result.shape)
    result.head()
    result_query_pair = result.with_columns(pl.lit(query_text).alias("query_text"))
    print(f"Time elapsed this query: {(end_time - start_time):.4f} seconds")
    return result_query_pair

In [ ]:
data_file = 'Queries.txt'
query_file = open(data_file, 'r')
queries = query_file.readlines()
query_file.close()

# query file is loaded, start asking questions one by one
pl.Config.set_fmt_str_lengths(3000)

# Ask our first query
Replies = make_query(queries[0].strip('\n').strip(' '))
Replies.shape
#print(Replies)
    

In [ ]:
# Let's ask the remaining queries

for i in range(1,len(queries)):
    replies = make_query(queries[i].strip('\n').strip(' '))
    
    Replies.extend(replies)

In [106]:
# Adding scoring columns for humans

raw_results = Replies.with_columns([pl.lit(None).alias("mo_label"),pl.lit(None).alias("kk_label"),pl.lit(None).alias("kp_label")
                      ,pl.lit(None).alias("dr_label"),pl.lit(None).alias("sr_label")])
print(raw_results.shape)
raw_results.head

#save to parquet file
raw_results.write_parquet('../config_%s/query_results_config_%s_metric_%s_full.parquet'%(config,config,met))

(1300, 23)
